In [1]:
#!/usr/bin/env python

from importlib import reload
# e.g. importlib.reload(Utilities) after changes made

import os
import sys
from pathlib import Path
import re
from difflib import SequenceMatcher
import shutil
import pandas as pd
from time import perf_counter
#---------------------------------------------------------------------
import MetricNames
import Utilities
import Utilities_df
import Utilities_xml
import Plot_Box_sns

In [2]:
def get_nist_files_in_test_dir(test_dir, sub_dirs_to_ignore):
    sub_dirs_to_search = Utilities.get_immediate_sub_dir_names(test_dir)
    sub_dirs_to_search = Utilities.remove_tagged_from_list(sub_dirs_to_search, sub_dirs_to_ignore)
    #-----
    found_nist_xmls = []
    # First, check for NIST files directly inside test_dir
    found_nist_xmls.extend(
        Utilities_xml.find_all_nist_paths(test_dir, 
                                          glob_pattern=r'*.xml', 
                                          regex_pattern=None, regex_ignore_case=False, recursive=False, 
                                          tags_to_ignore=['error', 'temporary', 'SPCA', 'report'])
    )
    # Next, search for NIST files in sub directories
    for sub_dir in sub_dirs_to_search:
        # NOTE: tags_to_ignore=['error', 'temporary', 'SPCA']+sub_dirs_to_ignore below
        #       This was to weed out, e.g. files in 
        #       \\milky-way\projects\BACOArchive\Waiting_For_Review\IQ-FAT\6040CTiX\Smiths_FAT_TRN_83375_SN_133076_20200610\MVY_ORT_TRN_83383_SN_040632_20202JUN11
        #       Not sure why this MVY_ORT_TRN_83383_SN_040632_20202JUN11 subdirectory is there to begin...?
        found_nist_xmls.extend(
            Utilities_xml.find_all_nist_paths(os.path.join(test_dir, sub_dir), 
                                              glob_pattern=r'**\*.xml', 
                                              regex_pattern=None, regex_ignore_case=False, recursive=True, 
                                              tags_to_ignore=['error', 'temporary', 'SPCA', 'report']+sub_dirs_to_ignore)
    )
    #-----
    test_dir_name = test_dir[test_dir.rfind(os.sep)+len(os.sep):]
    return (test_dir_name, found_nist_xmls)

In [3]:
def get_all_test_dirs_and_nist_xmls_in_base_dir(base_dir, sub_dirs_to_ignore=None, verbose=True):
    # sub_dirs_to_ignore are the subdirs within the test_dirs
    if sub_dirs_to_ignore is None:
        sub_dirs_to_ignore = ['Dosimeter Files', 'Image', '_Event Description']
    #-----
    test_dirs = Utilities.get_immediate_sub_dirs(base_dir)
    #-----
    found_test_dir_and_nist_xmls_tuples = []
    for test_dir in test_dirs:
        if verbose:
            print(test_dir)
        t1_start = perf_counter()
        found = get_nist_files_in_test_dir(test_dir, sub_dirs_to_ignore)
        found_test_dir_and_nist_xmls_tuples.append(found)
        t1_stop = perf_counter()
        if verbose:
            print(f"Elapsed time: {t1_stop-t1_start}")
            print(found[0])
            print(f"# nist xmls found = {len(found[1])}")
            print()
    return found_test_dir_and_nist_xmls_tuples

In [4]:
def move_nist_xml(test_dir_name, nist_xml_path_org, base_dir_new, make_empty_spca_dir=True):
    test_dir_new = os.path.join(base_dir_new, test_dir_name)
    assert(not os.path.exists(test_dir_new))
    test_dir_new = os.path.join(base_dir_new, test_dir_name, 'Results')
    #-----
    # Make NIST directory and SPCA directory (if instructed)
    nist_dir = os.path.join(test_dir_new, 'NISTResults')
    os.makedirs(nist_dir)
    if make_empty_spca_dir:
        os.makedirs(os.path.join(test_dir_new, 'SPCAResults'))
    #-----
    nist_xml_path_new = os.path.join(nist_dir, Path(nist_xml_path_org).name)
    shutil.copyfile(nist_xml_path_org, nist_xml_path_new)
    return (nist_xml_path_org, nist_xml_path_new)

In [5]:
def move_test_dir_and_nist_xmls(test_dir_and_nist_xmls, base_dir_new, make_empty_spca_dir=True):
    # Returns a list of tuples
    # There is one tuple for each nist_xml in test_dir_and_nist_xmls
    # and each tuple is (old_path, new_path)
    test_dir_name = test_dir_and_nist_xmls[0]
    nist_xmls = test_dir_and_nist_xmls[1]
    nist_xml_paths_org_new = []
    assert(len(nist_xmls)>-1)
    if len(nist_xmls)==0:
        print(f'No nist_xmls found for {test_dir_name}')
    elif len(nist_xmls)==1:
        nist_xml_paths_org_new.append(move_nist_xml(test_dir_name, nist_xmls[0], base_dir_new, make_empty_spca_dir))
    else:
        for i,nist_xml in enumerate(nist_xmls, start=1):
            nist_xml_paths_org_new.append(move_nist_xml(f'{test_dir_name}_{i}', nist_xml, base_dir_new, make_empty_spca_dir))
    return nist_xml_paths_org_new


In [6]:
def move_all_test_dirs_and_nist_xmls(test_dir_and_nist_xmls_tuples, base_dir_new, make_empty_spca_dir=True):
    return_summary = {}
    for test_dir_and_nist_xmls in test_dir_and_nist_xmls_tuples:
        nist_xml_paths_org_new = move_test_dir_and_nist_xmls(test_dir_and_nist_xmls, base_dir_new, make_empty_spca_dir)
        assert(test_dir_and_nist_xmls[0] not in return_summary)
        return_summary[test_dir_and_nist_xmls[0]] = nist_xml_paths_org_new
    return return_summary

In [7]:
def output_return_summary(return_summary, base_dir_org, save_file=None):
    if save_file is None:
        save_file = sys.stdout
    # set save_file = sys.stdout to output to screen instead of file
    if save_file != sys.stdout:
        save_file = open(save_file, 'w')
    #------------------------------------
    save_file.write('*'*100+'\n')
    save_file.write(f'Moving files from base_dir_org = {base_dir_org}\n')
    save_file.write('*'*100+'\n')
    for test_dir_name, list_of_nist_xml_paths_org_new in return_summary.items():
        save_file.write('-'*100+'\n')
        save_file.write(test_dir_name + '\n')
        if len(list_of_nist_xml_paths_org_new)==0:
            save_file.write('NO NIST XMLS FOUND!!!!!\n')
        else:
            save_file.write(f'# Nist found = {len(list_of_nist_xml_paths_org_new)}\n\n')
            for old_new_tuple in list_of_nist_xml_paths_org_new:
                assert(len(old_new_tuple)==2)
                save_file.write(f'Old path: {old_new_tuple[0]}\n')
                save_file.write(f'New path: {old_new_tuple[1]}\n\n')
        save_file.write('-'*100+'\n')        
    #------------------------------------
    if save_file != sys.stdout:
        save_file.close()

In [ ]:
move_errors_to_bad = False

if move_errors_to_bad:
    ort_dir_path = r'C:\Users\BUXTONJ\Documents\Analysis\LocalDataStorage\ForConnor_Data\ORT\NewSATBaseline\CT80'
    all_subdir_names = Utilities.get_immediate_sub_dir_names(ort_dir_path)
    all_found = Utilities.find_all_paths(ort_dir_path, r'**\*error*')
    all_found = [x for x in all_found if x.find('BAD')<0]

    for found in all_found:
        subdir_to_move_org = found[:found.find(os.sep, len(ort_dir_path+os.sep))]
        assert(Path(subdir_to_move_org).name in all_subdir_names)
        subdir_to_move_new = os.path.join(ort_dir_path, '_BAD', Path(subdir_to_move_org).name)
        shutil.move(subdir_to_move_org, subdir_to_move_new)

